# Thesis work--1

### Importing the libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import randrange, uniform

### Global variables and functions for tuning

In [3]:
FEATURES = 3
ROWS = 200
LOW_RANGE = 0
HIGH_RANGE = 10
NUMBER_OF_REGIONS = 3
SELECTION_X_GENERATOR = ['random_normal_dist','random_normal_range','set_manual_x']
SELECTED_GENERATOR = SELECTION_X_GENERATOR[0]
APPLY_FEATURE_SCALING = False
formula = lambda X: X[:, 0]**2 - X[:, 1]**2 + X[:, 1] - 1

### Generating random numbers using normal distribution

In [4]:
def rnd_norm():
    X = []
    for itr in range(FEATURES):
        X.append(np.random.normal(size=ROWS))
    X = np.asarray(X)
    X = X.reshape(ROWS,FEATURES)
    return X
    

### GENERATE NUMBER USING RANGES 

In [5]:
def rnd_rng():
    X = []
    for feat_ in range(FEATURES):
        temp_ = []
        for rows_ in range(ROWS):
            temp_.append(uniform(LOW_RANGE, HIGH_RANGE))
        X.append(temp_)
        temp_ = []
    X = np.asarray(X)
    X = X.reshape(ROWS,FEATURES)
    return X

### MANUALLY SET X

In [6]:
def mnl_x():
    X = [[4.77483186, 3.39440415, 57.05801446, 6.71629158],
 [6.61347886, 7.2255493,  122.77241787, 7.31532667],
 [4.5746061,  6.94376097, 1.57398063, 7.61596309],
 [3.99442041, 7.74271857, -22.18103276, 8.69864701],
 [7.74195296, 8.66056046, 1000.71418768, 8.66512174],
 [0.70227095, 8.48112974, 12.18087887, 9.98354661],
 [3.30624855, 1.28647579, 29.10406468, 8.96630542],
 [5.7062847,  7.72417331, 56.45891841, 5.15614429],
 [3.70369596, 1.79230683, 157.60607582, 9.4736015 ],
 [5.36628365, 9.68201719, 888.00342486, 2.75472882]]
    X = np.asarray(X)
    return X

### SELECT GENERATION METHOD

In [7]:
if 'random_normal_dist' == SELECTED_GENERATOR:
    X = rnd_norm()
elif 'random_normal_range' == SELECTED_GENERATOR:
    X = rnd_rng()
elif 'set_manual_x' == SELECTED_GENERATOR:
    X = mnl_x()

### Generating random numbers Y

In [9]:
Y = formula(X)

### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
if APPLY_FEATURE_SCALING:
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

### Applying PCA

In [12]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1)
#print(X_train)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
#print(X_train_pca)
#print(X_test_pca)
# print(np.shape(X_train),np.shape(X_test))
#print(explained_variance)

### Applying LDA

In [13]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# lda = LDA(n_components = None)
# X_train_lda = lda.fit_transform(X_train, y_train)
# X_test_lda = lda.transform(X_test)

### Applying Kernel PCA

In [14]:
from sklearn.decomposition import KernelPCA
# kpca = KernelPCA(n_components = 2, kernel = 'rbf')
# X_train_pca = kpca.fit_transform(X_train)
# X_test_pca = kpca.transform(X_test)

### Calculating DV

In [15]:
# sort X_train_pca and y_train by index
sorted_indexes = np.argsort(X_train_pca,axis=0)
sorted_x_train_pca = X_train_pca[sorted_indexes]
sorted_y_train = y_train[sorted_indexes]

# Finding Change of Slope
slope1 = []
slope2 = []
for itr in range(1,len(sorted_x_train_pca)):
    slope1.append((sorted_y_train[itr]-sorted_y_train[itr-1])/(sorted_x_train_pca[itr]-sorted_x_train_pca[itr-1]))
for itr in range(1,len(slope1)):
    slope2.append((slope1[itr]-slope1[itr-1])/(sorted_x_train_pca[itr]-sorted_x_train_pca[itr-1]))

# normalize slope2 
normalized_slope2 = (slope2-min(slope2))/(max(slope2)-min(slope2))

# Calculating Quantiles
normalized_slope2 = np.reshape(normalized_slope2,len(normalized_slope2))
quantile_ranges = pd.qcut(normalized_slope2,NUMBER_OF_REGIONS,labels=False,retbins=True)
quantile_ranges = quantile_ranges[1]

# Adding the difficult Vectors
difficult_points = {}
for q_ind in range(NUMBER_OF_REGIONS):
    low = quantile_ranges[q_ind]
    high = quantile_ranges[q_ind+1]
    difficult_points[q_ind] = []
    for n_ind in range(0,len(normalized_slope2)):
        if normalized_slope2[n_ind] >= low and normalized_slope2[n_ind] <= high:
            difficult_points[q_ind].append(n_ind)
print(difficult_points)

{0: [1, 9, 12, 15, 16, 19, 22, 23, 25, 27, 30, 32, 35, 39, 41, 48, 51, 53, 55, 57, 59, 62, 63, 65, 67, 72, 76, 78, 80, 85, 88, 90, 92, 94, 98, 101, 104, 105, 107, 108, 110, 111, 113, 115, 117, 119, 128, 129, 131, 136, 145, 148, 154], 1: [2, 3, 5, 6, 7, 8, 10, 20, 21, 26, 28, 31, 33, 40, 43, 44, 45, 47, 52, 60, 69, 70, 79, 82, 83, 95, 96, 99, 102, 116, 121, 122, 123, 124, 125, 126, 132, 133, 134, 135, 137, 139, 140, 141, 142, 143, 144, 147, 151, 152, 155, 157], 2: [0, 4, 11, 13, 14, 17, 18, 24, 29, 34, 36, 37, 38, 42, 46, 49, 50, 54, 56, 58, 61, 64, 66, 68, 71, 73, 74, 75, 77, 81, 84, 86, 87, 89, 91, 93, 97, 100, 103, 106, 109, 112, 114, 118, 120, 127, 130, 138, 146, 149, 150, 153, 156]}


### Training of converted PCA

In [16]:
from gplearn.genetic import SymbolicRegressor


In [17]:
est_gp = SymbolicRegressor(population_size=200,
                           generations=20, stopping_criteria=0.01,
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0)

In [18]:
final_train_X = np.reshape(sorted_x_train_pca,(len(sorted_x_train_pca),1))
final_train_Y = np.reshape(sorted_y_train,(len(sorted_y_train),1))
est_gp.fit(final_train_X,final_train_Y)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    46.36 361.42354401403026        7 1.5530114564811555 1.3279155394997557      4.26s
   1     11.3 2.1449814571210433        7 1.4645813660312463 2.123786353548935      4.77s
   2     5.98 2.6618250230268656        7 1.4131397092935032 2.5867612641886235      4.33s
   3     2.58 2.849348669230036        1 1.3987103379300407 2.755785864067473      3.51s
   4     1.32 1.5601020721261465        1 1.364511278936234 3.0635773950117327      2.86s
   5     1.34 1.5955040073548066        1 1.3583875424002438 3.118691023835642      2.41s
   6      2.3 1.7897387901203101        1 1.3878125536897128 2.8538659222304235      2.12s
   7     1.42 1.6534561153179161        1 1.3859383583391827 2.870733680385195      1.83s
   8     1.67 1.798814421750319  

SymbolicRegressor(const_range=(-1.0, 1.0),
         function_set=('add', 'sub', 'mul', 'div'), generations=20,
         init_depth=(2, 6), init_method='half and half', max_samples=0.9,
         metric='mean absolute error', n_jobs=1, p_crossover=0.7,
         p_hoist_mutation=0.05, p_point_mutation=0.1, p_point_replace=0.05,
         p_subtree_mutation=0.1, parsimony_coefficient=0.01,
         population_size=200, random_state=0, stopping_criteria=0.01,
         tournament_size=20, verbose=1, warm_start=False)

In [19]:
print(est_gp._program)

-0.667


### Training of unconverted PCA

In [28]:
est_gp = SymbolicRegressor(population_size=200,
                           generations=20, stopping_criteria=0.01,
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0)
temp_y_train = np.reshape(y_train,(len(y_train),1))
est_gp.fit(X_train,temp_y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     33.4 768.7899027033251        9 1.3260588075470194 1.5326098234895347      3.83s
   1     8.42 2.3877529025668456        9 1.2886609818682824 1.8691902545981676      4.19s
   2     8.99 2.1382212078498637       17 1.0528173767128024 1.092013531745478      4.37s
   3     9.73 2.1242941171236724       23 0.8786627214793072 1.5354577153083542      4.67s
   4    11.66 1.8207930031642678       17 0.9144762832729083 1.424311407128667      5.16s
   5    11.93 1.8921884834483313       11 0.9326980127651413 1.0462583265142236      5.33s
   6     9.94 2.2087883192419566        7 0.9519330807205336 1.832407433008849      5.07s
   7     8.82 1.7884369677683893       11 0.8781756993116657 1.5369591475955036      4.77s
   8     9.73 3.626443697289096

SymbolicRegressor(const_range=(-1.0, 1.0),
         function_set=('add', 'sub', 'mul', 'div'), generations=20,
         init_depth=(2, 6), init_method='half and half', max_samples=0.9,
         metric='mean absolute error', n_jobs=1, p_crossover=0.7,
         p_hoist_mutation=0.05, p_point_mutation=0.1, p_point_replace=0.05,
         p_subtree_mutation=0.1, parsimony_coefficient=0.01,
         population_size=200, random_state=0, stopping_criteria=0.01,
         tournament_size=20, verbose=1, warm_start=False)

In [29]:
print(est_gp._program)

sub(add(mul(X0, X0), -0.643), mul(sub(X1, 0.353), add(X1, -0.643)))
